In [1]:
%load_ext autoreload
%autoreload 2

import copy
import os
import sys
from pathlib import Path
import subprocess
from platformdirs import user_cache_dir
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import Draw, AllChem

import ms_pred.common as common
from ms_pred.dag_pred.iceberg_elucidation import candidates_from_pubchem, iceberg_prediction, load_real_spec, load_pred_spec, elucidation_over_candidates, plot_top_mols, explain_peaks, modi_finder, generate_buyable_report, load_global_config

/home/roger/miniforge3/envs/ms-main/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%cd /home/roger/ms-pred

/mnt/c/Users/runzh/OneDrive/Documents/2023/ms-pred


# Global configs
Please modify the the configs in `configs/iceberg/iceberg_elucidation.yaml` accordingly; for `python_path` on molgpu, you can either
1) set up your own ms-gen environment from source
2) use Mrunali's pre-built environment, after telling conda to look in her folder with `conda config --add envs_dirs /home/mrunali/miniconda3/envs`
3) Clone Mrunali's pre-built environment into your own environment with `conda --clone` and then source that instead.

In [3]:
config = load_global_config()
common.plot_utils.set_style()

The following cell enables SA score. See instructions in [the rdkik blog](https://greglandrum.github.io/rdkit-blog/posts/2023-12-01-using_sascore_and_npscore.html)
You can safely skip it if you are not setting ``sa_score=True`` in function ``plot_top_mols``

In [4]:
# sys.path.append(os.path.join(os.environ['CONDA_PREFIX'],'share','RDKit','Contrib'))
sys.path.append(os.path.join('/home/roger/miniforge3/envs/ms-main','share','RDKit','Contrib'))
from SA_Score import sascorer

## Foodome preprocessing

In [8]:
df = pd.read_csv('/home/roger/ms_collaborators/NEU-Foodome/NDM_compound_identifiers_mit.csv')
out_dir = Path('/home/roger/ms_collaborators/NEU-Foodome/foodome')
out_dir.mkdir(parents=True, exist_ok=True)

smiles = []
ikeys = []
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    smi = row['rdkit_desalted_isosmiles']
    if type(smi) != str:
        continue
    if len(smi) == 0:
        continue
    mol = Chem.MolFromSmiles(smi)
    if common.is_charged(mol, mol_type='mol') or common.has_separate_components(mol, mol_type='mol') or common.has_isotopes(mol, mol_type='mol') or common.has_unsupported_elems(mol, mol_type='mol') or common.mass_from_smi(smi) > 1500:
        continue
    else:
        mol = common.rm_stereo(mol, mol_type='mol')
        smiles.append(Chem.MolToSmiles(mol))
        ikeys.append(Chem.MolToInchi(mol))

_, idx = np.unique(ikeys, return_index=True)
smiles = np.array(smiles)[idx].tolist()
smiles = common.sanitize(smiles, 'smi')
print(f'length of smiles: {len(smiles)}')

/tmp/ipykernel_4620/1320738871.py:1: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/home/roger/ms_collaborators/NEU-Foodome/NDM_compound_identifiers_mit.csv')
100%|██████████| 133832/133832 [02:02<00:00, 1096.68it/s]


length of smiles: 84475


In [32]:
with open(out_dir / 'candidates.smi', 'w') as f:
    f.writelines('\n'.join(smiles))

In [17]:
# Run prediction. It's run remotely on molgpu
enu_config = copy.deepcopy(config)

result_path, pmz = iceberg_prediction(smiles, [45], **enu_config)

CUDA_VISIBLE_DEVICES=0 /home/roger/miniforge3/envs/ms-main/bin/python src/ms_pred/dag_pred/predict_smis.py \
               --batch-size 8 \
               --num-workers 6 \
               --dataset-labels /home/roger/.cache/ms-pred/iceberg-elucidation/af589f008e16eef2488a1ab6a8224a98/cands_df_iceberg_elucidation.tsv \
               --sparse-out \
               --sparse-k 100 \
               --max-nodes 100 \
               --threshold 0.0 \
               --gen-checkpoint /home/roger/ms-models/iceberg_results_20241111/dag_nist20/split_1_rnd1/version_0/best.ckpt \
               --inten-checkpoint /home/roger/ms-models/iceberg_results_20241111/dag_inten_nist20/split_1_rnd1/version_1/best.ckpt \
               --save-dir /home/roger/.cache/ms-pred/iceberg-elucidation/af589f008e16eef2488a1ab6a8224a98 \
               --gpu \
               --adduct-shift


Traceback (most recent call last):
  File "/mnt/c/Users/runzh/OneDrive/Documents/2023/ms-pred/src/ms_pred/dag_pred/predict_smis.py", line 21, in <module>
    import pytorch_lightning as pl
  File "/home/roger/miniforge3/envs/ms-main/lib/python3.9/site-packages/pytorch_lightning/__init__.py", line 30, in <module>
    from pytorch_lightning.callbacks import Callback  # noqa: E402
  File "/home/roger/miniforge3/envs/ms-main/lib/python3.9/site-packages/pytorch_lightning/callbacks/__init__.py", line 14, in <module>
    from pytorch_lightning.callbacks.base import Callback
  File "/home/roger/miniforge3/envs/ms-main/lib/python3.9/site-packages/pytorch_lightning/callbacks/base.py", line 25, in <module>
    from pytorch_lightning.utilities.types import STEP_OUTPUT
  File "/home/roger/miniforge3/envs/ms-main/lib/python3.9/site-packages/pytorch_lightning/utilities/types.py", line 26, in <module>
    from torchmetrics import Metric
  File "/home/roger/miniforge3/envs/ms-main/lib/python3.9/site-pa

KeyboardInterrupt: 

In [18]:
# parse spectrum
# result_path: path to hdf5 file
out_dir = Path('/home/roger/ms_collaborators/NEU-Foodome/foodome')
smiles, pred_specs, pred_frags = load_pred_spec(out_dir, False)
meta_spec_list = []
for smi, pred_spec in zip(smiles, pred_specs):
    pmass = common.mass_from_smi(smi) + common.ion2mass['[M+H]+']
    for ev, spec in pred_spec.items():
        spec[:, 1] *= 100
        meta = {
            'PEPMASS': pmass,
            'SMILES': smi,
            'IKEY': common.inchikey_from_smiles(smi),
            'COLLISION_ENERGY': ev,
        }
        meta_spec_list.append((meta, [('spec', spec)]))

mgf_outstr = common.build_mgf_str(meta_spec_list)

with open(out_dir / 'NDM_compound_pred.mgf', 'w') as f:
    f.write(mgf_outstr)


100%|██████████| 84425/84425 [00:20<00:00, 4135.00it/s]


# Known compounds 20250319

In [24]:
df = pd.read_csv('/home/roger/ms_collaborators/NEU-Foodome/foodb_selected_compounds.csv')

smiles = []
energies = []
ikeys = []
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    smi = row['canonical_smiles']
    ce = row['collision_energy(ev)']
    if type(smi) != str:
        continue
    if len(smi) == 0:
        continue
    mol = Chem.MolFromSmiles(smi)
    if common.is_charged(mol, mol_type='mol') or common.has_separate_components(mol, mol_type='mol') or common.has_isotopes(mol, mol_type='mol') or common.has_unsupported_elems(mol, mol_type='mol') or common.mass_from_smi(smi) > 1500:
        continue
    ikey = Chem.MolToInchi(mol)
    if ikey in ikeys:
        idx = ikeys.index(ikey)
        if ce not in energies[idx]:
            energies[idx].append(ce)
    else:
        mol = common.rm_stereo(mol, mol_type='mol')
        smiles.append(Chem.MolToSmiles(mol))
        ikeys.append(ikey)
        energies.append([ce])

_, idx = np.unique(ikeys, return_index=True)
smiles = np.array(smiles)[idx].tolist()
energies = [energies[i] for i in idx]
# smiles = common.sanitize(smiles, 'smi')
print(f'length of smiles: {len(smiles)}')

100%|██████████| 728/728 [00:01<00:00, 530.98it/s]


length of smiles: 688


In [25]:
enu_config = copy.deepcopy(config)

result_path, pmz = iceberg_prediction(smiles, energies, **enu_config)

CUDA_VISIBLE_DEVICES=0 /home/roger/miniforge3/envs/ms-main/bin/python src/ms_pred/dag_pred/predict_smis.py \
               --batch-size 8 \
               --num-workers 6 \
               --dataset-labels /home/roger/.cache/ms-pred/iceberg-elucidation/2175ada67dc4949c941e548e0f651216/cands_df_iceberg_elucidation.tsv \
               --sparse-out \
               --sparse-k 100 \
               --max-nodes 100 \
               --threshold 0.0 \
               --gen-checkpoint /home/roger/ms-models/iceberg_results_20241111/dag_nist20/split_1_rnd1/version_0/best.ckpt \
               --inten-checkpoint /home/roger/ms-models/iceberg_results_20241111/dag_inten_nist20/split_1_rnd1/version_1/best.ckpt \
               --save-dir /home/roger/.cache/ms-pred/iceberg-elucidation/2175ada67dc4949c941e548e0f651216 \
               --gpu \
               --adduct-shift


Global seed set to 42


2025-03-19 18:48:01,321 INFO: 
adduct_shift: true
batch_size: 8
binned_out: false
dataset_labels: /home/roger/.cache/ms-pred/iceberg-elucidation/2175ada67dc4949c941e548e0f651216/cands_df_iceberg_elucidation.tsv
dataset_name: null
debug: false
gen_checkpoint: /home/roger/ms-models/iceberg_results_20241111/dag_nist20/split_1_rnd1/version_0/best.ckpt
gpu: true
inten_checkpoint: /home/roger/ms-models/iceberg_results_20241111/dag_inten_nist20/split_1_rnd1/version_1/best.ckpt
max_nodes: 100
num_bins: 15000
num_workers: 6
save_dir: /home/roger/.cache/ms-pred/iceberg-elucidation/2175ada67dc4949c941e548e0f651216
seed: 42
sparse_k: 100
sparse_out: true
split_name: split_22.tsv
subset_datasets: none
threshold: 0.0
upper_limit: 1500

2025-03-19 18:48:01,527 INFO: Loaded gen / inten models from /home/roger/ms-models/iceberg_results_20241111/dag_nist20/split_1_rnd1/version_0/best.ckpt & /home/roger/ms-models/iceberg_results_20241111/dag_inten_nist20/split_1_rnd1/version_1/best.ckpt
2025-03-19 18:48:

  0%|          | 0/89 [00:00<?, ?it/s]

2025-03-19 18:48:08,149 INFO: There are 712 entries to process


100%|██████████| 89/89 [00:22<00:00,  3.88it/s]


2025-03-19 18:48:31,107 INFO: Program finished in: 29.791834592819214 seconds


In [26]:
smiles, pred_specs, pred_frags = load_pred_spec(result_path, False)
meta_spec_list = []
for smi, pred_spec in zip(smiles, pred_specs):
    pmass = common.mass_from_smi(smi) + common.ion2mass['[M+H]+']
    for ev, spec in pred_spec.items():
        spec[:, 1] *= 100
        meta = {
            'PEPMASS': pmass,
            'SMILES': smi,
            'IKEY': common.inchikey_from_smiles(smi),
            'COLLISION_ENERGY': ev,
        }
        meta_spec_list.append((meta, [('spec', spec)]))

mgf_outstr = common.build_mgf_str(meta_spec_list)

with open(out_dir / 'foodb_selected_compounds_pred.mgf', 'w') as f:
    f.write(mgf_outstr)


100%|██████████| 712/712 [00:00<00:00, 2708.90it/s]
